In [8]:
#Libraries used
import pandas as pd
import numpy as np

In [2]:
#Load databases from https://mrv.emsa.europa.eu/#public/emission-report

data18=pd.read_excel("2018-v258-30062021-EU MRV Publication of information.xlsx",index_col=0,skiprows=2)
data19=pd.read_excel("2019-v189-10082021-EU MRV Publication of information.xlsx",index_col=0,skiprows=2)
data20=pd.read_excel("2020-v25-01082021-EU MRV Publication of information.xlsx",index_col=0,skiprows=2)
data20=data20.rename(columns={"Time spent at sea [hours]": "Total time spent at sea [hours]", "Annual Time spent at sea [hours]": "Annual Total time spent at sea [hours]"})

In [4]:
#Combine databases
data=pd.concat([data18,data19,data20])

35921

In [6]:
#Removing ships that didn't travel
data['Annual average Fuel consumption per distance [kg / n mile]'] = pd.to_numeric(data['Annual average Fuel consumption per distance [kg / n mile]'], errors='coerce')
data=data.dropna(subset=["Annual average Fuel consumption per distance [kg / n mile]"])
data["Name"].count()

34565

In [9]:
#Create one unique method column
data['MethodA'] = np.where(data['A']=='Yes', 'A',"")
data["MethodB"]=np.where(data['B']=='Yes', 'B',"")
data["MethodC"]=np.where(data['C']=='Yes', 'C',"")
data["MethodD"]=np.where(data['D']=='Yes', 'D',"")
data= data.assign(Method = data.MethodA.astype(str)+data.MethodB.astype(str) +data.MethodC.astype(str) + data.MethodD.astype(str))


In [10]:
# Convert technical efficiency to numeric data
data["Technical efficiency (gCO₂/t·nm)"]=data["Technical efficiency"].str.replace(r"\D+",'')
data['Technical efficiency (gCO₂/t·nm)'] = pd.to_numeric(data['Technical efficiency (gCO₂/t·nm)'], errors='coerce')
data=data.dropna(subset=["Technical efficiency (gCO₂/t·nm)"])
data['Technical efficiency (gCO₂/t·nm)']=data['Technical efficiency (gCO₂/t·nm)']/100

In [12]:
#Calculating distance of ships
data["Distance travelled(n miles)"]=(data["Total fuel consumption [m tonnes]"]/data["Annual average Fuel consumption per distance [kg / n mile]"])*1000

In [13]:
#Calculating Speed of ships
data["Average speed (n miles/hour)"]=data["Distance travelled(n miles)"]/data["Annual Total time spent at sea [hours]"]

In [14]:
#Write cleaned database as a CSV file to be imported to Tableau
data.to_csv('EUShipping.csv')